To-do:
1. Текст был очищен только от одного мусорного элемента в качестве примера. Исследуйте данные через ноутбук или через веб-интерфейс BigQuery на предмет других мусорных элементов в тексте, которые не несут в себе никакого особого смысла, а только создают шум в данных. Доработайте функцию очистки тектосвых данных, чтобы в нее можно было передать список ненужного мусора и разом выполнялась очистка
2. Проведите стратифицировнную кросс-валидацию нейросетевого классификатора https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
3. Реверс-инжениринг нейронной сети (вектор - фразы - частотный анализ встречаемых слов - удаление или добавление в модель)

НЕ НАДО 3. Поэксперементируйте с гиперпараметрами нейросетевого классификатора, постарайтесь повысить качество его работы

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
4. Попробуйте использовать не Word2Vec для получения векторого представления текста, а TF-IDF преобразование http://zabaykin.ru/?p=558 http://nlpx.net/archives/57
5. Попробуйте использовать более тонко настриваемые алгоритмы нейросетей, например из этого видео https://www.youtube.com/watch?v=cPkH1k3U1c8 

In [1]:
import warnings
warnings.filterwarnings('ignore')

from google.oauth2 import service_account
import pandas_gbq 

import numpy as np
import pandas as pd
import math as mt
import datetime as dt

from langdetect import detect
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix, roc_auc_score
from gensim.models.word2vec import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve,auc
import matplotlib

In [2]:
CREDENTIALS = ''

In [3]:
#funtion for getting fresh data from DWH for workload model
"""[summary]
Funtion for getting fresh data from BigQuery for workload scoring model
[description]
Credentials - google service account object with credentials data for project
[example]
Input: Credentials = credentials_object
Output: description	                                        channel	 category	category_flag
        \nChat transcript:\nVisitor: I want to buy wit...	chat	 ps	        1
        \nChat transcript:\nVisitor: hell i had a prob...	chat	 ps	        1
        \nChat transcript:\nVisitor: لا استطيع الشراء ...	 chat	  ps	     1
"""
def getDwhData(Credentials):
    statement_bigquery_sql = " ".join(["select description, channel, case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 'ps'",
                                       "else 'other'",
                                       "end as category,",
                                       "case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 0",
                                       "else 1",
                                       "end as category_flag",
                                       "from `xsolla_summer_school.customer_support`",
                                       "where manual_category is not null and",
                                       "manual_category <> '' and",
                                       "description is not null and",
                                       "description <> '' and",
                                       "channel is not null and",
                                       "channel <> '' and",
                                       "channel in ('chat','facebook')"])
    
    dataframe_bigquery = pandas_gbq.read_gbq(statement_bigquery_sql,project_id='findcsystem', credentials=Credentials, dialect='standard')

    return dataframe_bigquery


"""[summary]
Function for transform text to lower case
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["text_1","text_2"]
"""
def lowerCase(Corpus):
    corpus = [i.lower().replace('\n',' ') for i in Corpus]
    return corpus

"""[summary]
Function for delete punctuation form text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
################################################Output: ["Text_1","Text_2"]
"""
def clearPunctuation(Corpus):
    corpus = [i.translate(str.maketrans('', '', string.punctuation)) for i in Corpus]
    return corpus

"""[summary]
Function for getting language of text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["en","ru"]
"""
def getTextLanguage(Corpus):
    txt_lang = []
    for txt in Corpus:
        try:
            lang = detect(txt)
            txt_lang.append(lang)
        except:
            lang = 'error'
            txt_lang.append(lang)
    
    return txt_lang


"""[summary]
Function for tokenization text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["word1 word2","word3 word4"]
Output: [["word1","word2"],["word3","word4"]]
"""  
def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 


"""[summary]
Function for clear text after garbage
[description]
Corpus - list or array object, with text data
Substr - string, regular expression
Garbage - list of string
[example]
Input: Corpus = [["word1","word2"],["word3","word4"]]
       Substr = r'word1'
Output: [["word2"],["word3","word4"]]
"""  
def clearTextAfterGarbage(Corpus,Substr):
    clear_corpus = []
    for text in Corpus:
        indexes = []
        text_len = len(text)
        try:
            for i in range(0,text_len):
                res = re.search(Substr,text[i])
                if res != None:
                    indexes.append(i)
                
            #delete garbage word from text
            for index in indexes:
                del text[index]
        
            clear_corpus.append(text)
        except:
            #clear_corpus.append("error")
            clear_corpus.append(text) # а то слишком много ошибок
        
    return clear_corpus

def clearTextAfterGarbageList2(Corpus,Garbage):
    clear_corpus = Corpus
    for Substr in Garbage:
        clear_corpus = clearTextAfterGarbage(clear_corpus,Substr)
    return clear_corpus    

def clearTextAfterGarbageList(Corpus,Garbage):
    clear_corpus = []
    for text in Corpus:
        for Substr in Garbage:
            indexes = []
            text_len = len(text)
            try:
                for i in range(0,text_len):
                    res = re.search(Substr,text[i])
                    if res != None:
                        indexes.append(i)
                
                #delete garbage word from text
                for index in indexes:
                    del text[index]
        
                clear_corpus.append(text)
            except:
                #clear_corpus.append("error")
                clear_corpus.append(text) # а то слишком много ошибок
        
    return clear_corpus

"""[summary]
Function for clear error after clearTextAfterGarbage(Corpus,Substr)
[description]
InsertDataFrame - pandas DataFrame
[example]
Input: InsertDataFrame = [{'description':['Text_1','error','Text_2'], field1':[1,2,3]}]}]
Output: [{'description':['Text_1','Text_2'], field1':[1,3]}]}]
""" 

def clearDescriptionError(InsertDataFrame):
    result = pd.DataFrame()
    result = InsertDataFrame[InsertDataFrame.description != 'error']
    return result

"""[summary]
Build word vector by using pre-trained Word2Vec model
[description]
Size - lenght of vector
Word2Vec_Model - gensim object
"""  
def buildWordVector(Text,Size,Word2Vec_Model):
    vec = np.zeros(Size).reshape((1,Size))
    count = 0.

    for word in Text:
        try:
            vec += Word2Vec_Model[word].reshape((1,Size))
            count += 1.
        except KeyError:
            continue
    
    if count != 0:
        vec /= count
    
    return vec

def countWords(InsertDataFrame):
    df_count = pd.DataFrame({'text':[], 'count':[]})

    for row in InsertDataFrame.description:
        for text in row:            
            if text in df_count.text.values:
                index = df_count[df_count.text == text].index
                df_count.loc[index,'count'] = df_count.loc[index,'count']+1
            else:
                df_count = df_count.append(pd.DataFrame({'text':text, 'count':[int(1)]}))
    return df_count

RAWDATA 

In [4]:
#getting data from dwh
SupportRawDataframe = getDwhData(CREDENTIALS)
SupportRawDataframe.shape
#SupportRawDataframe.head(10)
#SupportRawDataframe.tail(10)

Downloading: 100%|███████████████████| 23450/23450 [00:14<00:00, 1610.04rows/s]


(23450, 4)

DATA PREPROC

In [5]:
#transform text to lower case
corpus = SupportRawDataframe.description
corpus.astype('str')

corpus = clearPunctuation( lowerCase(corpus) )

#getting language for text corpus
corpus_lang = getTextLanguage(corpus)

In [6]:
#new dataframe with texts in lower case, without /n symbol and with lang for text
SupportRawDataframe['description'] = corpus
SupportRawDataframe['lang'] = corpus_lang
#SupportRawDataframe.head(10)
#SupportRawDataframe.tail(10)

In [7]:
#getting only en texts
SupportDataframe_eng = SupportRawDataframe[SupportRawDataframe.lang == 'en'][:]
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,chat transcript visitor i want to buy with pa...,chat,ps,0,en
1,chat transcript visitor hell i had a problem ...,chat,ps,0,en
2,chat transcript visitor لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript visitor im having trouble wit...,chat,ps,0,en
4,chat transcript visitor hi ana hello how can ...,chat,ps,0,en
...,...,...,...,...,...
23445,chat transcript visitor hi i made a pruchase ...,chat,other,1,en
23446,chat transcript visitor hi how long will it t...,chat,other,1,en
23447,chat transcript visitor i bought playerunknow...,chat,other,1,en
23448,chat transcript visitor good day i took the w...,chat,other,1,en


In [8]:
#text tekenization
tokenization = textToTokens(SupportDataframe_eng.description)
SupportDataframe_eng['description'] = tokenization

In [9]:
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[chat, transcript, visitor, i, want, to, buy, ...",chat,ps,0,en
1,"[chat, transcript, visitor, hell, i, had, a, p...",chat,ps,0,en
2,"[chat, transcript, visitor, لا, استطيع, الشراء...",chat,ps,0,en
3,"[chat, transcript, visitor, im, having, troubl...",chat,ps,0,en
4,"[chat, transcript, visitor, hi, ana, hello, ho...",chat,ps,0,en
...,...,...,...,...,...
23445,"[chat, transcript, visitor, hi, i, made, a, pr...",chat,other,1,en
23446,"[chat, transcript, visitor, hi, how, long, wil...",chat,other,1,en
23447,"[chat, transcript, visitor, i, bought, playeru...",chat,other,1,en
23448,"[chat, transcript, visitor, good, day, i, took...",chat,other,1,en


In [10]:
texts = list(SupportDataframe_eng.description)

In [11]:
#cleaning text after garbage
GarbageList = ['chat','transcript','visitor', 'hi', 'hello', 'i', 'as', 'for', 'to', 'a', 'the', 'you', 'this', 'or', 'u', 'me', 'hi', 'im', 'your', 'is', 'on', 'its', 'it', 'into']
#tests_clear = clearTextAfterGarbage(texts,r'chat')
#tests_clear = clearTextAfterGarbage(texts,r'transcript')
#tests_clear = clearTextAfterGarbage(texts,r'visitor')
tests_clear = clearTextAfterGarbageList2(texts,GarbageList)

In [12]:
SupportDataframe_eng['description'] = tests_clear
SupportDataframe_eng = clearDescriptionError(SupportDataframe_eng)
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[and, a, problem, us, error, code, a, screensh...",chat,ps,0,en
1,"[hell, a, yesterday, people, i, got, refund, i...",chat,ps,0,en
2,"[لا, استطيع, الشراء, ومعلومات, الشراء, صحيحة, ...",chat,ps,0,en
3,"[xsolla, 4, code, a, pymaent, ana, ana, we, th...",chat,ps,0,en
4,"[how, can, pay, i, credit, with, credit, canno...",chat,ps,0,en
...,...,...,...,...,...
23445,"[a, 25, dollars, and, my, wallet, specified, w...",chat,other,1,en
23446,"[how, my, friend, may, with, contain, and, 31,...",chat,other,1,en
23447,"[battlegrounds, request, refund, provide, 9, s...",chat,other,1,en
23448,"[good, package, to, and, the, way, refund, so,...",chat,other,1,en


In [13]:
'''df_count = pd.DataFrame({'text':[], 'count':[]})
df_count0 = pd.DataFrame({'text':[]})

for row in SupportDataframe_eng.description:
    for text in row:
        df_count0 = df_count0.append(pd.DataFrame({'text':[text]}))               
        if text in df_count.text.values:
            index = df_count[df_count.text == text].index
            df_count.loc[index,'count'] = df_count.loc[index,'count']+1
        else:
            df_count = df_count.append(pd.DataFrame({'text':text, 'count':[int(1)]}))
            
import seaborn as sns
import matplotlib.pyplot as plt

df_count2 = df_count.sort_values('count', ascending=False).iloc[:100,:]
#fig = plt.figure(figsize=(100,10))
#plt.bar(df_count2.text, df_count2['count'])
df_count2.head(50)            
#df_count'''

"df_count = pd.DataFrame({'text':[], 'count':[]})\ndf_count0 = pd.DataFrame({'text':[]})\n\nfor row in SupportDataframe_eng.description:\n    for text in row:\n        df_count0 = df_count0.append(pd.DataFrame({'text':[text]}))               \n        if text in df_count.text.values:\n            index = df_count[df_count.text == text].index\n            df_count.loc[index,'count'] = df_count.loc[index,'count']+1\n        else:\n            df_count = df_count.append(pd.DataFrame({'text':text, 'count':[int(1)]}))\n            \nimport seaborn as sns\nimport matplotlib.pyplot as plt\n\ndf_count2 = df_count.sort_values('count', ascending=False).iloc[:100,:]\n#fig = plt.figure(figsize=(100,10))\n#plt.bar(df_count2.text, df_count2['count'])\ndf_count2.head(50)            \n#df_count"

TRAIN TEST SPLIT DATAFRAMES

In [14]:
#list of unique categories
unique_categories = np.unique(SupportDataframe_eng.category)
unique_categories

array(['other', 'ps'], dtype=object)

In [15]:
descriptions = SupportDataframe_eng['description']
categories = SupportDataframe_eng['category_flag']
XTrain,XTest,YTrain,YTest = train_test_split(descriptions,
                                             categories,
                                             stratify = categories,
                                             test_size = 0.2,
                                             random_state = 40)

FROM TEXTS TO VECTORS

In [33]:
#initialize Word2Vec model for embedding words to vectors
NDim = 100
Imdb_w2v = Word2Vec(size = NDim,min_count = 10)
#Imdb_w2v.build_vocab(XTrain)
Imdb_w2v.build_vocab(descriptions)

In [34]:
Imdb_w2v.train(XTrain,total_examples = Imdb_w2v.corpus_count,epochs = Imdb_w2v.epochs)

(2200185, 3266335)

In [35]:
#embedding training messages to vectors for neutral classifier
TrainVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTrain])

In [36]:
TrainVecs

array([[-0.43586432, -0.3959808 , -0.01990268, ...,  0.66250673,
        -0.39381805,  0.31354541],
       [-0.21275633, -0.67683196,  0.20512251, ...,  0.60520251,
        -0.25326288,  0.28399209],
       [-0.19479879, -0.39241365,  0.08012048, ...,  0.74108996,
        -0.28729453,  0.22459949],
       ...,
       [-0.42994174, -0.54227297,  0.03541817, ...,  0.61616622,
        -0.41158503,  0.06789512],
       [-0.52070432, -0.62364154,  0.00704202, ...,  0.92890784,
        -0.28629432,  0.46993134],
       [-0.01574008, -0.40637667,  0.15611375, ...,  0.54203011,
        -0.12883063,  0.19917874]])

In [37]:
Imdb_w2v.train(XTest, total_examples = Imdb_w2v.corpus_count, epochs = Imdb_w2v.epochs)

(541733, 807410)

In [38]:
TestVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTest])

In [39]:
TestVecs

array([[-0.14535375, -0.51181917,  0.00454482, ...,  0.65385855,
        -0.07724295,  0.41435156],
       [ 0.05663901, -1.20904526,  0.12860918, ...,  0.50547934,
         0.36272812,  0.0336782 ],
       [-0.03271364, -0.33371408,  0.14655118, ...,  0.61399629,
        -0.21952226, -0.07903461],
       ...,
       [-0.20673065, -0.3120412 ,  0.2128201 , ...,  0.61218973,
         0.07239278, -0.01282447],
       [-0.49525105, -0.11586727, -0.08771099, ...,  0.30783819,
        -0.03695738,  0.33467935],
       [-0.04624643, -0.37248702,  0.00967495, ...,  0.74459834,
        -0.29364556,  0.32198195]])

CROSSVALIDATION AND BUILD CLASSIFIER

In [40]:
TextClassifier = MLPClassifier(hidden_layer_sizes = (20,10), max_iter = 1000, random_state = 40)

In [41]:
Scores = cross_val_score(TextClassifier, TrainVecs, YTrain, cv = 5)
print(Scores)
print(np.mean(Scores))

[0.88639876 0.88485317 0.87789799 0.88330757 0.88060278]
0.8826120556414219


In [42]:
TextClassifier.fit(TrainVecs,YTrain)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=40, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [43]:
pred = TextClassifier.predict(TestVecs)
print(confusion_matrix(YTest,pred))
print(classification_report(YTest,pred))

[[ 371  282]
 [ 116 2467]]
              precision    recall  f1-score   support

           0       0.76      0.57      0.65       653
           1       0.90      0.96      0.93      2583

    accuracy                           0.88      3236
   macro avg       0.83      0.76      0.79      3236
weighted avg       0.87      0.88      0.87      3236



Кросс-валидация

In [27]:
skf = StratifiedKFold(n_splits=2, random_state=40, shuffle=False)
MLPClassifier_sum = int(0)

NDim = 100

for train_index, test_index in skf.split(descriptions, categories):
    Imdb_w2v = Word2Vec(size = NDim,min_count = 10)
    #Imdb_w2v.build_vocab(descriptions.loc[descriptions.index.intersection(train_index)])
    Imdb_w2v.build_vocab(descriptions)
    Imdb_w2v.train(descriptions.loc[descriptions.index.intersection(train_index)],total_examples = Imdb_w2v.corpus_count,epochs = Imdb_w2v.epochs)
    TrainVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in descriptions.loc[descriptions.index.intersection(train_index)]])
    Imdb_w2v.train(descriptions.loc[descriptions.index.intersection(test_index)], total_examples = Imdb_w2v.corpus_count, epochs = Imdb_w2v.epochs)
    TestVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in descriptions.loc[descriptions.index.intersection(test_index)]])
    
    TextClassifier.fit(TrainVecs,categories.loc[categories.index.intersection(train_index)])
    MLPClassifier_sum += roc_auc_score(categories.loc[categories.index.intersection(test_index)], TextClassifier.predict(TestVecs))
    print(classification_report(categories.loc[categories.index.intersection(test_index)],TextClassifier.predict(TestVecs)))
MLPClassifier_total = MLPClassifier_sum / skf.get_n_splits()
print('mean roc_auc', MLPClassifier_total)

              precision    recall  f1-score   support

           0       0.75      0.33      0.45      2400
           1       0.66      0.92      0.77      3357

    accuracy                           0.67      5757
   macro avg       0.70      0.62      0.61      5757
weighted avg       0.70      0.67      0.64      5757

              precision    recall  f1-score   support

           0       0.37      0.60      0.46       864
           1       0.91      0.80      0.86      4594

    accuracy                           0.77      5458
   macro avg       0.64      0.70      0.66      5458
weighted avg       0.83      0.77      0.79      5458

mean roc_auc 0.6636018962525408


Зависит ли точность модели от симметричности категорий рандомной выборки?

Но тут рассмотрим слова в ошибочно отнесённых записях

In [28]:
# index of wrong predictions
error_zero = list()
error_one = list()
for i in range(YTest.size):
    if YTest.iloc[i] == 0:
        if YTest.iloc[i] != pred[i]:
            error_zero.append(YTest.index[i])
    else:
        if YTest.iloc[i] != pred[i]:
            error_one.append(YTest.index[i])        

In [29]:
df_error_zero = SupportDataframe_eng[SupportDataframe_eng.index.isin(error_zero)]
df_error_one = SupportDataframe_eng[SupportDataframe_eng.index.isin(error_one)]

In [30]:
words_error_zero = countWords(df_error_zero)
words_error_one = countWords(df_error_one)

In [31]:
words_error_zero.head(50)

,text,count
0,how,15820.0
0,i,15820.0
0,cant,15820.0
0,have,15820.0
0,1049,15820.0
0,sorry,15820.0
0,reasons,15820.0
0,for,15820.0
0,please,15820.0
0,will,15820.0


In [32]:
words_error_one.head(50)

,text,count
0,problem,3348.0
0,joel,3348.0
0,how,3348.0
0,can,3348.0
0,i,3348.0
0,pay,3348.0
0,314980885,3348.0
0,and,3348.0
0,might,3347.0
0,to,3347.0
